### 日志数据处理

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# 读取 Excel 文件
file_path = './data/raw_data.xlsx'  # 替换为你的 Excel 文件路径
sheet_name = 'Sheet1'  # 指定工作表名称，如果不指定，则默认读取第一个工作表

# 使用 pandas 的 read_excel 函数读取数据
data = pd.read_excel(file_path, sheet_name=sheet_name)

#### 数据的一些特征

1. 一共35万条数据，一共28万个用户，说明大部分用户其实都只挂过一次号(时间跨度大概是一个月)
   - 这些只挂过一次号的基本可以认为是正常用户
   - 先找出重复挂号的用户，分析其特征

In [52]:
# 找出重复挂号的用户的ID
duplicated_rows = data[data['患者ID'].duplicated(keep=False)]
duplicated_rows = duplicated_rows.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])

In [53]:
# 保存
output_file = "./data/repeat_records.xlsx"
duplicated_rows.to_excel(output_file, sheet_name="duplicate", index=False)

In [10]:
MAX_DUP = 6
# 筛选出重复次数大于5的和无重复的
value_counts = data['患者ID'].value_counts()
# 筛选出重复次数大于5的行
mass_duplicated = data[data['患者ID'].isin(value_counts[value_counts > MAX_DUP].index)]
mass_duplicated = mass_duplicated.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])
print("大量重复的行: ", len(mass_duplicated))

大量重复的行:  14443


In [11]:
# 筛选无重复的行
unique_rows = data[data['患者ID'].isin(value_counts[value_counts == 1].index)]
unique_rows = unique_rows.sort_values(by=['患者ID', '订单创建时间'], ascending=[True, True])
print("无重复的行: ", len(unique_rows))

无重复的行:  154603


In [12]:
print(data.columns)

Index(['ID', '省份', 'APPID', 'IP_ADDRESS', '订单创建时间', '患者ID', '患者创建时间', '就诊日期',
       '就诊科室名称', '医生姓名', '状态', '商户订单号'],
      dtype='object')


In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

In [ ]:

# 列名称
columns = ['ID', '省份', 'APPID', 'IP_ADDRESS', '订单创建时间', '患者ID', '患者创建时间', '就诊日期',
           '就诊科室名称', '医生姓名', '状态', '商户订单号']

# 分类特征列
categorical_features = ['省份', 'APPID', 'IP_ADDRESS', '就诊科室名称', '医生姓名', '患者ID', '商户订单号']

# 时间特征列
time_features = ['订单创建时间', '患者创建时间', '就诊日期']

# 数值特征列
numeric_features = ['ID']

# 处理分类特征
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 处理数值特征
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# 预处理管道
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 异常检测模型
model = IsolationForest()

# 训练模型
model.fit(preprocessor.fit_transform(data[columns]))

# 预测异常
predictions = model.predict(preprocessor.transform(data[columns]))
data['Anomaly'] = predictions


            ID   省份                             APPID       IP_ADDRESS  \
0            1   北京  9f2fd531c80a48849af7a3c180a8d267   221.216.34.188   
1            2   北京  f297332c9d424acdb5b97aa7b6922e3a    223.104.40.76   
2            3   河北  0bc9f59db31844aabe1b266229518655    36.143.73.105   
3            4   北京  71b15132c79d4ae18c9a5213750656e9  118.247.172.109   
4            5   北京  3b94345c553447a697d3c44092807cf1   223.104.38.155   
...        ...  ...                               ...              ...   
348205  348206   北京  d1a16219220b4af685fbf90232f0f421   120.244.38.164   
348206  348207   北京  02f90c86930848de937aa5075a6459ec   223.104.41.243   
348207  348208   北京  80c2ac86b5604a468e45fcd64f775432    223.72.179.14   
348208  348209   北京  fa721bad2a704c7c8f93bd52edf76a96   120.244.182.37   
348209  348210  黑龙江  1c4e2c4e72e84c3f9740b2d8cf2b69cf     1.56.172.120   

                     订单创建时间     患者ID               患者创建时间        就诊日期 就诊科室名称  \
0       2024-04-09 16:59:59  T0

In [17]:
# 打印结果
print(data['Anomaly'])

0         1
1         1
2         1
3         1
4         1
         ..
348205    1
348206    1
348207    1
348208    1
348209    1
Name: Anomaly, Length: 348210, dtype: int32
